# About
* **Author**: Adil Rashitov (adil@wastelabs.co)
* **Created at**: 07.09.2022

In [ ]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Multiprocessing for Mac / Linux
import platform
platform.system()
if platform.system() == 'Darwin':
    from multiprocess import Pool
else:
    from multiprocessing import Pool

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)


# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

# GLOBAL VARS
from pathlib import Path
import sys
PROJECT_DIR = os.getcwd().rsplit('/', 1)[0]
SRC_DIR = PROJECT_DIR 


if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)

# Main

In [ ]:

body = {
    "task_id": "BusinessAdmin_e0245692-50b2-4b5b-947a-4803ca2b722a_20220906_235833",
    "assigned_stops": [
        {
            "stop_id": "78250043525562469",
            "route_id": "W01",
            "route_sequence": 2,
            "latitude": 51.491754,
            "longitude": -0.0796267,
            "duration_seconds": 0,
            "open_time": "00:00:00",
            "close_time": "23:59:59"
        },
        {
            "stop_id": "306261",
            "route_id": "W01",
            "route_sequence": 1,
            "latitude": 51.5121,
            "longitude": -0.1201,
            "duration_seconds": 300,
            "open_time": "00:00:00",
            "close_time": "23:59:59"
        },
        {
            "stop_id": "78250043525562469",
            "route_id": "W01",
            "route_sequence": 0,
            "latitude": 51.491754,
            "longitude": -0.0796267,
            "duration_seconds": 0,
            "open_time": "00:00:00",
            "close_time": "23:59:59"
        },
    ],
    "assigned_routes": [
        {
            "route_id": "W01",
            "vehicle_type": "Van",
            "shift_start_time": "08:00:00"
        },
    ]
}



In [ ]:
from typing import Literal
from datetime import date
from faker import Faker


class RouteFaker(Faker):

    def route_stops(
            self,
            route_id: str,
            n_stops: int,
            date: date,
            pivot_lat: float = 51.5072,
            pivot_lon: float = 0.1276,
            radius: float = 0.1):
        stops = []
        for stop_id in range(n_stops):
            stops.append({
                "latitude": float(self.coordinate(pivot_lat, radius)),
                "longitude": float(self.coordinate(pivot_lon, radius)),
                "duration_seconds": int(faker.random_int(min=60, max=600)),
                "open_time": f"{date} 00:00:00",
                "close_time": f"{date} 23:59:59",
                "stop_id": str(faker.random_number()),
                "route_sequence": stop_id,
                "route_id": route_id,
            })
        return stops

    def factory_message_body(self):
        _date = self.date_between_dates(date_start=date(2020, 1, 1), date_end=date(2022, 10, 5))
        route_id_van = faker.aba()
        route_id_bicycle = faker.aba()
        task_id = f"BusinessAdmin_{_date}-{route_id_van}-{route_id_bicycle}"
        n_stops_van = faker.random_int(min=7, max=25)
        n_stops_bicycle = faker.random_int(min=7, max=25)

        return {
            "task_id": task_id,
            "assigned_stops": (
                faker.route_stops(route_id=route_id_van, n_stops=n_stops_van, date=_date) +
                faker.route_stops(route_id=route_id_bicycle, n_stops=n_stops_bicycle, date=_date)
            ),
            "assigned_routes": [
                {
                    "route_id": route_id_van,
                    "vehicle_type": "Van",
                    "shift_start_time": f"{_date} 08:00:00"
                },
                {
                    "route_id": route_id_bicycle,
                    "vehicle_type": "Bicycle",
                    "shift_start_time": f"{_date} 08:00:00"
                },
            ]            
        }


In [ ]:
!rm -rf ../data/01_raw/fake_message_bodies/
!mkdir -p ../data/01_raw/fake_message_bodies/

In [ ]:
faker = RouteFaker()

In [ ]:
faker.aba

In [ ]:
import json


def export_json(file_id: int, body: dict):
    with open(f'../data/01_raw/fake_message_bodies/{file_id}.json', 'w', encoding='utf-8') as f:
        json.dump(body, f)


for i in range(0, 200):
    logging.info(f"Start processing sample: {i}")
    body = faker.factory_message_body()
    export_json(file_id=i, body=body)

In [ ]:
pd.DataFrame(body["assigned_routes"])

In [ ]:
pd.DataFrame(body["assigned_stops"])